In [1]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [2]:
# Reproducability
np.random.seed(42)
keras.utils.set_random_seed(42)

In [3]:
# Input image shape
input_shape = (500, 500, 3)

# Data directories
base_dir = '/content/drive/MyDrive/ML/Data'
train_dir = base_dir + '/Train'
valid_dir = base_dir + '/Valid'
test_dir = base_dir + '/Test'

In [4]:
# Data Augmentation
train_datagen = keras.Sequential([
    layers.Rescaling(1./255),
    layers.RandomZoom(0.2),
    layers.RandomRotation(0.1),
    layers.RandomFlip("horizontal"),
    layers.RandomContrast(0.1),
    layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
    layers.RandomShear(0.1)
])

valid_datagen = keras.Sequential([
    layers.Rescaling(1./255)
])

In [ ]:
# Data generators
BATCH_SIZE = 32 # 16 used for model 2 due to memory limits
train_ds_original = keras.utils.image_dataset_from_directory(
    train_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(input_shape[0], input_shape[1]),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=True
)

valid_ds_original = keras.utils.image_dataset_from_directory(
    valid_dir,
    labels='inferred',
    label_mode='categorical',
    image_size=(input_shape[0], input_shape[1]),
    interpolation='nearest',
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds_original.class_names

# Apply data augmentation
train_ds = train_ds_original.map(lambda x, y: (train_datagen(x, training=True), y))
valid_ds = valid_ds_original.map(lambda x, y: (valid_datagen(x, training=False), y))

# Display number of batches
print(f"Number of training batches: {tf.data.experimental.cardinality(train_ds)}")
print(f"Number of validation batches: {tf.data.experimental.cardinality(valid_ds)}")

In [ ]:
# Model1
model1 = Sequential([
    keras.Input(shape=input_shape),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(3, activation='softmax')
])

model1.summary()

In [7]:
# Compile
model1.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training
epochs = 20

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)

history_model1 = model1.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds,
    callbacks=[early_stopping]
)

In [ ]:
import pickle

# Save history1
history_save_path_model1 = '/content/drive/MyDrive/ML/training_history_model1.pkl'

with open(history_save_path_model1, 'wb') as f:
    pickle.dump(history_model1.history, f)

print(f"Training history saved to: {history_save_path_model1}")

In [ ]:
# Save trained model1
model_save_path1 = '/content/drive/MyDrive/ML/trained_model1.keras'

model1.save(model_save_path1)

print(f"Model saved to: {model_save_path1}")

In [ ]:
plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history_model1.history['accuracy'], label='Training Accuracy')
plt.plot(history_model1.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history_model1.history['loss'], label='Training Loss')
plt.plot(history_model1.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Model2
model2 = Sequential([
    keras.Input(shape=input_shape),
    layers.Conv2D(64, (3, 3), activation='relu'), # Increased filters
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'), # Increased filters
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(256, (3, 3), activation='relu'), # Increased filters
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.3), # Reduced dropout layer rate
    layers.Dense(3, activation='softmax')
])

model2.summary()

In [7]:
# Compile
model2.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5), # Adjusted learning rate for stability
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Training
epochs = 20

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True
)

history_model2 = model2.fit(
    train_ds,
    epochs=epochs,
    validation_data=valid_ds,
    callbacks=[early_stopping]
)

In [ ]:
# Save history2
history_save_path_model2 = '/content/drive/MyDrive/ML/training_history_model2.pkl'

with open(history_save_path_model2, 'wb') as f:
    pickle.dump(history_model2.history, f)

print(f"Training history for model 2 saved to: {history_save_path_model2}")

In [ ]:
# Save model2
model_save_path2 = '/content/drive/MyDrive/ML/trained_model2.keras'

model2.save(model_save_path2)

print(f"Model 2 saved to: {model_save_path2}")

In [ ]:
plt.figure(figsize=(12, 4))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history_model2.history['accuracy'], label='Training Accuracy')
plt.plot(history_model2.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model 2 Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history_model2.history['loss'], label='Training Loss')
plt.plot(history_model2.history['val_loss'], label='Validation Loss')
plt.title('Model 2 Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# Save class names
class_names_save_path = '/content/drive/MyDrive/ML/class_names.pkl'

with open(class_names_save_path, 'wb') as f:
    pickle.dump(class_names, f)

print(f"Class names saved to: {class_names_save_path}")